# Router Query Engine

Routers serve as specialized modules designed to process a user's query and select from a set of predefined "choices," characterized by their metadata.

There are two primary types of core router modules:

1. **LLM Selectors:** These selectors present the available choices as a text prompt, utilizing the LLM text completion endpoint for decision-making.

2. **Pydantic Selectors:** Here, choices are passed in the form of Pydantic schemas to a function-calling endpoint. The results are then returned as Pydantic objects.

## Setup

In [1]:
# NOTE: This is ONLY necessary in jupyter notebook.
# Details: Jupyter runs an event-loop behind the scenes.
#          This results in nested event-loops when we start an event-loop to make async queries.
#          This is normally not allowed, we use nest_asyncio to allow it for convenience.
import nest_asyncio

nest_asyncio.apply()

In [11]:
import logging
import sys

# Set up the root logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Set logger level to INFO

# Clear out any existing handlers
logger.handlers = []

# Set up the StreamHandler to output to sys.stdout (Colab's output)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)  # Set handler level to INFO

# Add the handler to the logger
logger.addHandler(handler)

from llama_index.core import VectorStoreIndex, SummaryIndex, SimpleDirectoryReader, StorageContext
from llama_index.core import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from IPython.display import display, HTML

In [3]:
import os 
from dotenv import load_dotenv, find_dotenv

In [4]:
load_dotenv('/home/santhosh/Projects/courses/Pinnacle/.env')

True

In [5]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

## Download Data

In [7]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-31 13:43:56--  https://raw.githubusercontent.com/run-llama/llama_index/9607a05a923ddf07deee86a56d386b42943ce381/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  70.8KB/s    in 1.0s    

2024-03-31 13:43:58 (70.8 KB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



## Load data

In [9]:
# load documents
documents = SimpleDirectoryReader("data/paul_graham").load_data()

nodes = Settings.node_parser.get_nodes_from_documents(documents, chunk_size=1024)

## Define Summary Index and Vector Index over Same Data

In [14]:
embed_model = OpenAIEmbedding(model='text-embedding-3-small')

In [15]:
# Summary Index for summarization questions
summary_index = SummaryIndex(nodes, embed_model=embed_model)

# Vector Index for answering specific context questions
vector_index = VectorStoreIndex(nodes, embed_model=embed_model)

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


## Define Query Engines.

1. Summary Index Query Engine.
2. Vector Index Query Engine.

In [16]:
# Summary Index Query Engine
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# Vector Index Query Engine
vector_query_engine = vector_index.as_query_engine()

## Build summary index and vector index tools

In [18]:
from llama_index.core.tools.query_engine import QueryEngineTool

# Summary Index tool
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description="Useful for summarization questions related to Paul Graham eassy on What I Worked On.",
)

# Vector Index tool
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description="Useful for retrieving specific context from Paul Graham essay on What I Worked On.",
)

## Define Router Query Engine

Various selectors are at your disposal, each offering unique characteristics.

Pydantic selectors, supported exclusively by gpt-4-0613 and the default gpt-3.5-turbo-0613, utilize the OpenAI Function Call API. Instead of interpreting raw JSON, they yield pydantic selection objects.

On the other hand, LLM selectors employ the LLM to generate a JSON output, which is then parsed to query the relevant indexes.

For both selector types, you can opt to route to either a single index or multiple indexes.

## PydanticSingleSelector

Use the OpenAI Function API to generate/parse pydantic objects under the hood for the router selector.

In [20]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors.llm_selectors import LLMSingleSelector, LLMMultiSelector
from llama_index.core.selectors.pydantic_selectors import PydanticMultiSelector, PydanticSingleSelector

In [21]:
# Create Router Query Engine
query_engine = RouterQueryEngine(
    selector=PydanticSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)

In [22]:
response = query_engine.query("What is the summary of the document?")

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 0: The choice is specifically focused on summarization questions related to Paul Graham's essay on What I Worked On..
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [23]:
display(HTML(f'<p style="font-size:14px">{response.response}</p>'))

## LLMSingleSelector

Utilize OpenAI (or another LLM) to internally interpret the generated JSON and determine a sub-index for routing.

In [24]:
# Create Router Query Engine
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
)

In [25]:
response = query_engine.query("What is the summary of the document?")

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 0: The summary of the document would be best captured by summarization questions related to Paul Graham's essay on What I Worked On..
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [26]:
display(HTML(f'<p style="font-size:14px">{response.response}</p>'))

In [27]:
response = query_engine.query("What did Paul Graham do after RICS?")

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 1: Useful for retrieving specific context from Paul Graham essay on What I Worked On..
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [28]:
display(HTML(f'<p style="font-size:14px">{response.response}</p>'))

## PydanticMultiSelector

If you anticipate queries being directed to multiple indexes, it's advisable to use a multi-selector. This selector dispatches the query to various sub-indexes and subsequently aggregates the responses through a summary index to deliver a comprehensive answer.

## Let's create a simplekeywordtable index and corresponding tool.

In [30]:
from llama_index.core import SimpleKeywordTableIndex

keyword_index = SimpleKeywordTableIndex(nodes)

keyword_query_engine = keyword_index.as_query_engine()

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    description="Useful for retrieving specific context using keywords from Paul Graham essay on What I Worked On.",
)

## Build a router query engine.

In [31]:
query_engine = RouterQueryEngine(
    selector=PydanticMultiSelector.from_defaults(),
    query_engine_tools=[
        vector_tool,
        keyword_tool,
        summary_tool
    ],
)

In [32]:
# This query could use either a keyword or vector query engine, so it will combine responses from both
response = query_engine.query(
    "What were noteable events and people from the authors time at Interleaf and YC?"
)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 0: This choice is useful for retrieving specific context from the Paul Graham essay on What I Worked On, which can help in identifying notable events and people from the author's time at Interleaf and YC..
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Selecting query engine 1: This choice is useful for retrieving specific context using keywords from the Paul Graham essay on What I Worked On, which can aid in identifying notable events and people from the author's time at Interleaf and YC..
> Starting query: What were noteable events and people from the authors time at Interleaf and YC?
query keywords: ['people', 'noteable', 'events', 'interleaf', 'authors', 'time', 'yc']
> Extracted keywords: ['people', 'interleaf', 'time', 'yc']
HTTP Request: POST https://api.openai.com/v1/chat

In [33]:
display(HTML(f'<p style="font-size:14px">{response.response}</p>'))